## 32단계: 고차 미분(구현 편)

> 이번 단계에서는 고차 미분을 해낼 수 있도록 DeZero를 변경하겠습니다. 이전 단계에서 이야기한 대로 역전파 시 수행되는 계산에 대해서도 Variable 인스턴스를 사용해 계산 그래프를 만들면 됩니다.

### 32.1 새로운 DeZero로!

기존 `core_simple.py`를 `core.py`로 복사한 뒤, `Variable` 클래스를 다음과 같이 수정한다.

In [ ]:
class Variable:
    ...
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            # self.grad = np.ones_like(self.data)
            self.grad = Variable(np.ones_like(self.data))  # grad is now a Variable
        ...

### 32.2 함수 클래스의 역전파

이제 다음 함수 클래스들의 `backward` 메서드를 확인하고, 필요시 수정한다.
- Add
- Mul
- Neg
- Sub
- Div
- Pow

In [ ]:
class Add(Function):
    ...
    
    def backward(self, gy):
        # Add 클래스는 입력을 그대로 전달하기 때문에 수정할게 없다.
        return gy, gy

In [ ]:
class Mul(Function):
    ...

    def backward(self, gy):
        # gy가 이제 ndarray가 아닌 Variable이므로
        # inputs에서 ndarray를 꺼내지 않고 그대로 사용한다.
        
        # x0, x1 = self.inputs[0].data, self.inputs[1].data
        x0, x1 = self.inputs
        return gy * x1, gy * x0

In [ ]:
class Neg(Function):
    ...
    
    def backward(self, gy):
        # 입력 Variable의 부호를 바꿔주므로 변경 사항이 없다.
        return -gy

In [ ]:
class Sub(Function):
    ...
    
    def backward(self, gy):
        # 변경 사항 없음
        return gy, -gy

In [ ]:
class Div(Function):
    ...
    
    def backward(self, gy):
        # Mul 클래스와 마찬가지.
        
        # x0, x1 = self.inputs[0].data, self.inputs[1].data
        x0, x1 = self.inputs
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [ ]:
class Pow(Function):
    ...
    
    def backward(self, gy):
        # Mul 클래스와 마찬가지.
        
        # x = self.inputs[0].data
        x = self.inputs
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

### 32.3 역전파를 더 효율적으로(모드 추가)

18단계에서의 '역전파 비활성 모드'와 비슷하게, 2차 미분을 계산할 일이 없다면 역전파 계산을 '역전파 비활성 모드'로 실행하도록 하자.

In [ ]:
class Variable:
    ...
    
    # def backward(self, retain_grad=False):
    def backward(self, retain_grad=False, create_graph=False):
        if self.grad is None:
            self.grad = Variable(np.ones_like(self.data))
        
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                heappush(funcs_heap, f)
                seen_set.add(f)
        
        add_func(self.creator)
        
        while funcs_heap:
            f = heappop(funcs_heap)
            gys = [output.grad for output in f.outputs]
            
            # create_graph == True인 경우에만 계산 그래프를 생성한다.
            with using_config('enable_backprop', create_graph):
                gxs = f.backward(*gys)  # target 1
                if not isinstance(gxs, tuple):
                    gxs = (gxs,)
                
                for x, gx in zip(f.inputs, gxs):
                    if x.grad is None:
                        x.grad = gx
                    else:
                        x.grad = x.grad + gx  # target 2
                    
                    if x.creator is not None:
                        add_func(x.creator)
                
                if not retain_grad:
                    for y in f.outputs:
                        y.grad = None

`create_graph=False`로 기본값이 설정된 이유는 보통 1차 미분만 사용하는 경우가 압도적으로 많기 때문.

### 32.4 `__init__.py` 변경

이것으로 새로운 DeZero의 핵심이 완성되었다. 이제 `core.py`가 기초를 갖췄으므로, `__init__.py`에서 `is_simple_core = False`로 설정한다.

In [ ]:
# __init__.py

# is_simple_core = True
is_simple_core = False  # 수정

if is_simple_core:
    from dezero.core_simple import (
        Variable,
        Function,
        using_config,
        no_grad,
        as_variable,
        as_array,
        setup_variable,
    )

else:
    from dezero.core import (
        Variable,
        Function,
        using_config,
        no_grad,
        as_variable,
        as_array,
        setup_variable,
    )


setup_variable()

이것으로 이번 단계를 마친다. 다음 단계에서는 새로은 DeZero를 사용하여 고차 미분을 자동으로 계산해보자.